# importing the MLBox

In [1]:
from mlbox.preprocessing import *
from mlbox.optimisation import *
from mlbox.prediction import *

/home/datageek/anaconda2/lib/python2.7/site-packages/mlbox/preprocessing/drift/__init__.py:9: UserWarning: ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'.
  warnings.warn("ipCluster is starting. Please wait 30 sec and check in terminal that 'the engines appear to have started successfully'.")
Using Theano backend.
/home/datageek/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
paths = ["train.csv","test.csv"]
target_name="SalePrice"

# reading and cleaning all files 

In [3]:
rd = Reader(sep = ',')
df = rd.train_test_split(paths, target_name)


reading csv : train.csv ...
cleaning data...
CPU time: 0.277560949326 seconds

reading csv : test.csv ...
cleaning data...
CPU time: 0.430703878403 seconds

number of common features : 80

gathering and crunching for train and test datasets
reindexing for train and test datasets
dropping training duplicates
dropping constant variables on training set

number of categorical features : 43
number of numerical features : 37
number of training samples : 1460
number of test samples : 1459

Top sparse features (% missing values on train set):
PoolQC         99.5
MiscFeature    96.3
Alley          93.8
Fence          80.8
FireplaceQu    47.3
dtype: float64

task : regression


In [4]:
dft = Drift_thresholder()
df = dft.fit_transform(df)


computing drifts...
CPU time: 0.981590032578 seconds

Top 10 drifts

('Id', 0.9996570644718793)
('GarageYrBlt', 0.52930157759352303)
('Exterior1st', 0.52586411001119482)
('TotalBsmtSF', 0.52389869237891595)
('2ndFlrSF', 0.52123329243075833)
('Exterior2nd', 0.52107292808122652)
('BsmtFullBath', 0.52018890605818879)
('GrLivArea', 0.5197186371334116)
('TotRmsAbvGrd', 0.5194699326175769)
('WoodDeckSF', 0.51865392689173118)

deleted variables : ['Id']

dumping drift coefficients into directory : save
drift coefficients dumped


# tuning

In [5]:
mape = make_scorer(lambda y_true, y_pred: 100*np.sum(np.abs(y_true-y_pred)/y_true)/len(y_true), greater_is_better=False, needs_proba=False)
opt = Optimiser(scoring = mape, n_folds = 3)

In [6]:
opt.evaluate(None, df)

No parameters set. Default configuration is tested

##################################################### testing hyper-parameters... ############################################################

>>> NA ENCODER :{'numerical_strategy': 'mean', 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'label_encoding'}

>>> ESTIMATOR :{'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'colsample_bytree': 0.8, 'silent': True, 'max_depth': -1, 'learning_rate': 0.05, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'n_estimators': 500, 'subsample': 0.9, 'reg_lambda': 0, 'subsample_freq': 1, 'max_bin': 255, 'objective': 'regression', 'min_split_gain': 0, 'seed': 0, 'min_child_samples': 10, 'boosting_type': 'gbdt'}


MEAN SCORE : make_scorer(<lambda>, greater_is_better=False) = -16.5388085094
VARIANCE : 2.00049776054 (fold 1 = -19.3362884829, fold 2 = -15.5055357349, fold 3 = -14.7746013104)
CPU time: 1.98886799812 seconds



-16.538808509423966

In [7]:
space = {
    
        'ne__numerical_strategy':{"search":"choice",
                                 "space":[0]},
        'ce__strategy':{"search":"choice",
                        "space":["label_encoding","random_projection", "entity_embedding"]}, 
        'fs__threshold':{"search":"uniform",
                        "space":[0.01,0.3]},    
        'est__max_depth':{"search":"choice",
                                  "space":[3,4,5,6,7]}
    
        }

best = opt.optimise(space, df,15)


##################################################### testing hyper-parameters... ############################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'random_projection'}

>>> FEATURE SELECTOR :{'threshold': 0.11800533974130921, 'strategy': 'l1'}

>>> ESTIMATOR :{'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'colsample_bytree': 0.8, 'silent': True, 'learning_rate': 0.05, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'n_estimators': 500, 'subsample': 0.9, 'reg_lambda': 0, 'subsample_freq': 1, 'min_child_samples': 10, 'max_bin': 255, 'objective': 'regression', 'min_split_gain': 0, 'seed': 0, 'max_depth': 7, 'boosting_type': 'gbdt'}



/home/datageek/anaconda2/lib/python2.7/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)



MEAN SCORE : make_scorer(<lambda>, greater_is_better=False) = -16.5979217673
VARIANCE : 2.04512479245 (fold 1 = -19.4539929126, fold 2 = -15.5647866061, fold 3 = -14.7749857833)
CPU time: 4.25830888748 seconds


##################################################### testing hyper-parameters... ############################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'entity_embedding'}

>>> FEATURE SELECTOR :{'threshold': 0.18109153505020234, 'strategy': 'l1'}

>>> ESTIMATOR :{'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'colsample_bytree': 0.8, 'silent': True, 'learning_rate': 0.05, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'n_estimators': 500, 'subsample': 0.9, 'reg_lambda': 0, 'subsample_freq': 1, 'min_child_samples': 10, 'max_bin': 255, 'objective': 'regression', 'min_split_gain': 0, 'seed': 0, 'max_depth': 5, 'boosting_type': 'gbdt'}


MEAN SCORE : mak


MEAN SCORE : make_scorer(<lambda>, greater_is_better=False) = -16.7815941693
VARIANCE : 2.0056464797 (fold 1 = -19.5798218248, fold 2 = -15.7841869429, fold 3 = -14.9807737403)
CPU time: 5.88804197311 seconds


##################################################### testing hyper-parameters... ############################################################

>>> NA ENCODER :{'numerical_strategy': 0, 'categorical_strategy': '<NULL>'}

>>> CA ENCODER :{'strategy': 'random_projection'}

>>> FEATURE SELECTOR :{'threshold': 0.2580180424442389, 'strategy': 'l1'}

>>> ESTIMATOR :{'num_leaves': 31, 'reg_alpha': 0, 'subsample_for_bin': 50000, 'colsample_bytree': 0.8, 'silent': True, 'learning_rate': 0.05, 'nthread': -1, 'min_child_weight': 5, 'strategy': 'LightGBM', 'n_estimators': 500, 'subsample': 0.9, 'reg_lambda': 0, 'subsample_freq': 1, 'min_child_samples': 10, 'max_bin': 255, 'objective': 'regression', 'min_split_gain': 0, 'seed': 0, 'max_depth': 7, 'boosting_type': 'gbdt'}


MEAN SCORE : make

In [8]:
prd = Predictor()
prd.fit_predict(best,df)


fitting the pipeline...
CPU time: 1.31591105461 seconds

predicting...
CPU time: 0.209532022476 seconds

top 10 predictions :

   SalePrice_predicted
0        132369.726218
1        141944.351496
2        167870.013821
3        180968.771399
4        204228.908576
5        175222.131177
6        169116.571074
7        168311.500489
8        183939.248489
9        132881.731769

dumping predictions into directory : save


<mlbox.prediction.predictor.Predictor instance at 0x7f034c165d40>